In [1]:
import os
import polars as pl
import requests

In [2]:
df = pl.read_parquet(os.path.join("data","projevy.parquet")).sort(by='datum')

In [3]:
df.filter(pl.col('mluvci').str.contains('Babiš')).select(pl.col(['mluvci','mluvci_id'])).unique()

mluvci,mluvci_id
str,str
"""Předseda vlády ČR Andrej Babiš""","""https://www.vlada.cz/cz/clenov…"
"""Poslanec Andrej Babiš""","""/sqw/detail.sqw?id=6150"""
"""Poslanec Miloš Babiš""","""/sqw/detail.sqw?id=6151"""
"""Poslankyně Andrea Babišová""","""/sqw/detail.sqw?id=6428"""
"""Místopředseda vlády ČR a minis…","""http://www.vlada.cz/cz/clenove…"


In [4]:
len(df)

801985

In [5]:
len(df.filter(pl.col('mluvci_id').str.contains('/sqw/detail')))

477038

In [6]:
df.filter(pl.col('mluvci').str.contains('Marek Benda')).select(pl.col(['mluvci','mluvci_id'])).unique()

mluvci,mluvci_id
str,str
"""Poslanec Marek Benda""","""/sqw/detail.sqw?id=5723&sfo=7"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=3616"""
"""Poslanec Marek Benda""","""9"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=10693"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=7050"""
…,…
"""Poslanec Marek Benda (z místa)""","""/sqw/detail.sqw?id=5723&sfo=7"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=1915"""
"""Pan poslanec Marek Benda""","""/sqw/detail.sqw?id=5723&sfo=7"""


In [7]:
mluvci_raw = df.select(pl.col("mluvci_id")).unique().to_series().to_list()

In [8]:
len(mluvci_raw)

2174

In [9]:
import random

In [10]:
random.sample(mluvci_raw, 20)

['/sqw/detail.sqw?id=6110',
 '/sqw/p.sqw?P=572',
 '/sqw/detail.sqw?id=5888',
 '/sqw/detail.sqw?id=6773',
 '/sqw/detail.sqw?id=275',
 '/sqw/hlasy.sqw?G=3546',
 '/sqw/detail.sqw?id=6604',
 'http://www.vlada.cz/cz/clenove-vlady/miroslava-kopicova-57177/',
 '/sqw/hlasy.sqw?G=4322',
 '/sqw/detail.sqw?id=67&sfo=6',
 '/ff/27/34/01.htm',
 '/sqw/detail.sqw?id=5969',
 '/sqw/detail.sqw?id=31&sfo=6',
 '/sqw/detail.sqw?o=3&id=165',
 '/sqw/detail.sqw?id=5267',
 '/sqw/detail.sqw?id=5478',
 '/sqw/detail.sqw?id=6503',
 '/sqw/p.sqw?P=620',
 '/sqw/detail.sqw?id=5635&sfo=6',
 '/sqw/detail.sqw?id=5738']

In [11]:
kam = "downloads/poslanectvo"

In [12]:
os.makedirs(kam, exist_ok=True)

In [13]:
def stahni_poslance(url):
    if not '/sqw/detail.sqw?' in url:
        return None
    else:
        id = url.split('id=')[-1].split('&')[0]
        for obdobi in range(0,10):
            soubor = f'{id}_{obdobi}.html'
            if obdobi == 0:
                print(f'Stahuji id {id}')
            if soubor not in stazene:
                try:
                    if 'psp.cz' not in url:
                        url = 'https://www.psp.cz' + url
                    if 'https://' not in url:
                        url = 'https://' + url
                    response = requests.get(url)
                    if response.status_code == 200:
                        with open(os.path.join(kam, soubor), 'w+', encoding='utf-8') as poslancoobdobi:
                            poslancoobdobi.write(response.text)
                except:
                    print(f"chyba: {url}")

In [14]:
stazene = set(os.listdir(kam))

print(f"{len(stazene)} souborů staženo")

for m in mluvci_raw:
    stahni_poslance(m)

3942 souborů staženo
Stahuji id 5280
Stahuji id 6730
Stahuji id 6182
Stahuji id 317
Stahuji id 5290
Stahuji id 6163
Stahuji id 6679
Stahuji id 6541
Stahuji id 5615
Stahuji id 5255
Stahuji id 367
Stahuji id 5751
Stahuji id 31
Stahuji id 5971
Stahuji id 5513
Stahuji id 6299
Stahuji id 5365
Stahuji id 377
Stahuji id 6478
Stahuji id 295
Stahuji id 382
Stahuji id 6044
chyba: https://www.psp.cz/sqw/detail.sqw?id=6044
Stahuji id 5276
Stahuji id 122
Stahuji id 392
Stahuji id 111
Stahuji id 388
Stahuji id 310
Stahuji id 5715
Stahuji id 6725
Stahuji id 5257
Stahuji id 6434
Stahuji id 413
Stahuji id 6476
Stahuji id 358
Stahuji id 6229
Stahuji id 379
Stahuji id 6436
Stahuji id 6284
Stahuji id 377
Stahuji id 253
Stahuji id 253
Stahuji id 5766
Stahuji id 5457
Stahuji id 5459
Stahuji id 5273
chyba: https://http://www.psp.cz/sqw/detail.sqw?id=5273
chyba: https://http://www.psp.cz/sqw/detail.sqw?id=5273
chyba: https://http://www.psp.cz/sqw/detail.sqw?id=5273
chyba: https://http://www.psp.cz/sqw/detail.

TypeError: argument of type 'NoneType' is not iterable